In [1]:
import pandas as pd
import numpy as np
from datetime import date
import datetime as DT
import pickle
import re
from IPython.core.display import display, HTML
import statsmodels
import statsmodels.api as sm
import sklearn
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
import xgboost
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')
import plotly
import plotly.figure_factory as ff
import plotly.express as px

In [2]:
print(pd.__version__)
print(np.__version__)
print(statsmodels.__version__)
print(sklearn.__version__)
print(xgboost.__version__)
print(plotly.__version__)
print(re.__version__)

1.2.0
1.19.5
0.12.2
0.23.2
1.3.1
4.12.0
2.2.1


In [3]:
'''
reading registration and transaction level data with derived features
'''

reg_derived_file = 'output/registration_output.csv'
txn_derived_file = 'output/transaction_output.csv'
final_model_pickle = 'pickle_files/xgb_model.pickle'
xgboost_train_output_file = 'output/xgboost_train_output.csv'
xgboost_test_output_file = 'output/xgboost_test_output.csv'

In [4]:
df_reg = pd.read_csv(reg_derived_file)

In [5]:
df_reg = df_reg.reset_index(drop=True)

In [6]:
df_reg = df_reg.rename(columns = {'Email': 'Email_Reg'})

In [7]:
reg_data_dtypes = df_reg.dtypes.to_frame().reset_index()
reg_data_dtypes.columns = ['VAR_NAME', 'DTYPE']
reg_data_dtypes

,VAR_NAME,DTYPE
0,FirstName,object
1,MiddleName,object
2,LastName,object
3,Dob,object
4,Gender,object
...,...,...
108,CountryMatching,float64
109,StateMatching,float64
110,LocalityMatching,float64
111,ZipcodeMatch-UserEntered&IP,float64


In [8]:
reg_data_dtypes.to_csv('intermediate_files/registration_derived_signals_dtypes.csv', index = False)

In [9]:
df_txn = pd.read_csv(txn_derived_file)

In [10]:
df_txn = df_txn.reset_index(drop=True)

In [11]:
df_txn = df_txn.rename(columns = {'Email': 'Email_Txn'})

In [12]:
txn_data_dtypes = df_txn.dtypes.to_frame().reset_index()
txn_data_dtypes.columns = ['VAR_NAME', 'DTYPE']
txn_data_dtypes

,VAR_NAME,DTYPE
0,Txn ID,int64
1,Transaction Type,object
2,Purchase Time,object
3,Amount,int64
4,Currency Code,int64
...,...,...
97,Transactions_In_One_Minute,int64
98,Transactions_Per_User_Per_Hr,int64
99,Transactions_Per_User_Per_Day,int64
100,Transactions_Per_User_Per_Month,int64


In [13]:
txn_data_dtypes.to_csv('intermediate_files/transaction_derived_signals_dtypes.csv', index = False)

In [14]:
df_reg['Mobile'].dtype

dtype('int64')

In [15]:
df_txn['Mobile Number'].dtype

dtype('int64')

In [16]:
# df_reg['Mobile'] = df_reg['Mobile'].astype(int)

In [17]:
df_merged = pd.merge(df_txn, df_reg, left_on = 'Mobile Number', right_on = 'Mobile', how = 'left', suffixes=('_left','_right'))

In [18]:
df_merged = df_merged.reset_index(drop=True)

In [19]:
df_merged.shape

(1485, 215)

In [20]:
df_merged.columns.to_list()

['Txn ID',
 'Transaction Type',
 'Purchase Time',
 'Amount',
 'Currency Code',
 'Transaction Status',
 'Batch ID',
 'Invoice ID',
 'Auth Code',
 'Approval Code',
 'RRN Number',
 'Success / Failure',
 'Caller Details',
 'Caller LoginId',
 'Merchant ID',
 'Tid',
 'Merchant',
 'Dynamic Merchant Name',
 'Merchant Customer ID',
 'Merchant Reference Number',
 'Root Txn Id',
 'Original Txn Id',
 'Order Description',
 'Client Ip',
 'Device Category',
 'PG Interface Detail',
 'PG Error Code',
 'PG Details',
 'Acquiring Error Code',
 'Acquiring Error Detail',
 'Card Type Code',
 'Card Issued Country Code',
 'Association Name',
 'Domestic/International Card Flag',
 'Credit/Debit Card Flag',
 'Card Number',
 'Name On Card',
 'Email_Txn',
 'Mobile Country Code',
 'Mobile Number',
 'Abuse status',
 'Ext1',
 'ECI',
 'card Bin',
 'Settlement Bin',
 'OverAll Status',
 'OverAll Transaction Type',
 'Acquirer Reference Number',
 'Presentment Type',
 'Presentment Status',
 'Amount_Range',
 'Purchase_Time_C

In [21]:
mod_data_dtypes = df_merged.dtypes.to_frame().reset_index()
mod_data_dtypes.columns = ['VAR_NAME', 'DTYPE']
mod_data_dtypes

,VAR_NAME,DTYPE
0,Txn ID,int64
1,Transaction Type,object
2,Purchase Time,object
3,Amount,int64
4,Currency Code,int64
...,...,...
210,CountryMatching,float64
211,StateMatching,float64
212,LocalityMatching,float64
213,ZipcodeMatch-UserEntered&IP,float64


In [22]:
mod_data_dtypes.to_csv('intermediate_files/modeling_data_dtypes.csv', index = False)

In [23]:
'''
set drop columns list
'''

drop_columns = ['Purchase Time',
'Batch ID',
'Invoice ID',
'Auth Code',
'Approval Code',
'RRN Number',
'Caller Details',
'Caller LoginId',
'Merchant ID',
'Tid',
'Merchant',
'Dynamic Merchant Name',
'Merchant Customer ID',
'Merchant Reference Number',
'Root Txn Id',
'Original Txn Id',
'Order Description',
'Client Ip',
'PG Interface Detail',
'PG Error Code',
'PG Details',
'Acquiring Error Code',
'Acquiring Error Detail',
'Card Number',
'Name On Card',
'Email_Txn',
'Mobile Number',
'Ext1',
'OverAll Transaction Type',
'Acquirer Reference Number',
'Presentment Type',
'Purchase_Time_Conv',
'Txn_Min',
'Txn_Hour',
'Txn_Date',
'Txn_Month',
'Txn_Year',
'EmailBody_Txn',
'SpclChar_InEmail_Txn',
'FirstName',
'MiddleName',
'LastName',
'Dob',
'Gender',
'UserId',
'Email_Reg',
'Mobile',
'ProgramId',
'ProgramName',
'PostalCode',
'LastBadLoginTryTime',
'LastLoginTime',
'LastPinChangeTime',
'PresentLoginTime',
'RegisteredOn',
'Reg Referral Code',
'Reg Referral Prefix',
'Ref PC AC Number',
'Reg Client IP',
'Device Id',
'Sim Id',
'Meta Data',
'FirstNameLower',
'MiddleNameLower',
'LastNameLower',
'FullName',
'RegMonth',
'RegDay',
'DobCleaned',
'DobMonth',
'DobDay',
'LastLoginTime(Sec)',
'LastLoginTime(Hour)',
'LastLoginTime(Min)',
'RegisteredOn(Hour)',
'RegisteredOn(Min)',
'LoginTimeInHour',
'LoginTimeInMin',
'Domain',
'EmailBody',
'SpclChar_InEmail',
'NumChar_InEmail',
'Email_Text',
'IPBlock1',
'IPBlock2',
'IPBlock3',
'EnglishWords_Txn',
'EnglishWords_Reg']

In [24]:
df = df_merged.drop(drop_columns, axis = 1)

In [25]:
df_merged_original = df_merged.copy()

In [26]:
df.shape

(1485, 127)

In [27]:
df = df.rename(columns={'Txn ID': 'Txn_ID'})

In [28]:
dv_encoding = {'Non Abuse': 0, 'Abuse': 1}
df['Abuse status'] = df['Abuse status'].map(dv_encoding)

In [29]:
df['Abuse status'].value_counts()

0    780
1    705
Name: Abuse status, dtype: int64

In [30]:
'''
univariate analysis
'''
def univariate_analysis(df, min_cov = 5):
    output = []
    include = ['float64', 'int64', 'object','int32','float32','datetime64[ns]']
    #Description of all the variables 
    nunique = pd.DataFrame(list(zip(list(df.columns), list(df.nunique()))), 
               columns =['variables', 'nunique'])
    uni_analysis = df.describe([.25, .50, .75, .90, .95], include).T.reset_index().rename(columns = {'index':'variables'})
    uni_analysis = uni_analysis.merge(nunique, how='left', on=['variables'])
    #Calculating the coverage for each variable 
    uni_analysis['coverage'] = uni_analysis['count']/df.shape[0]*100
    uni_analysis.drop(columns =['top','freq','unique'], inplace=True)
    #Filtering the variables where the coverage is less than min_cov
    drop_list = list(uni_analysis['variables'][(uni_analysis['coverage']<min_cov)|(uni_analysis['nunique'] == 1)])  
    #summary dataset
    output.append(uni_analysis)
    #drop_list
    output.append(drop_list)
    return output

In [31]:
#calling above created function 
op = univariate_analysis(df) 

In [32]:
#coverage for each varibale
display(HTML(op[0].to_html()))

,variables,count,mean,std,min,25%,50%,75%,90%,95%,max,nunique,coverage
0,Txn_ID,1485.0,451030672.278114,28740276.186627,400128376.000000,427030669.000000,453654658.000000,475634335.000000,490089354.400000,495052474.400000,499962010.000000,1485,100.000000
1,Transaction Type,1485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,100.000000
2,Amount,1485.0,4192.454545,2234.772654,505.000000,2278.000000,4147.000000,6054.000000,7187.600000,7588.800000,24356.000000,1349,100.000000
3,Currency Code,1485.0,356.000000,0.000000,356.000000,356.000000,356.000000,356.000000,356.000000,356.000000,356.000000,1,100.000000
4,Transaction Status,1485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,100.000000
5,Success / Failure,1485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,100.000000
6,Device Category,1485.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,100.000000
7,Card Type Code,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.491582
8,Card Issued Country Code,1485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,100.000000
9,Association Name,1485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,100.000000


In [33]:
if 'Abuse status' in op[1]:
    op[1].remove('Abuse status') 

In [34]:
#list of variables that are required to dropped based on univariate analysis 
op[1]

['Transaction Type',
 'Currency Code',
 'Transaction Status',
 'Success / Failure',
 'Device Category',
 'Card Type Code',
 'Domestic/International Card Flag',
 'Mobile Country Code',
 'OverAll Status',
 'Presentment Status',
 'N_Txn_Per_Dynamic_Merchant_Name_Per_Month',
 'N_Txn_Per_Dynamic_Merchant_Name_Per_Hour',
 'N_Txn_Per_Card_Per_Hour',
 'N_Txn_Per_Card_Per_Day',
 'N_Txn_Per_Card_Per_Month',
 'N_Txn_With_Same_Amount',
 'Freq_Of_Each_Amount_Transacted',
 'Spcl_Char_Count_Txn',
 'Total_Txn',
 'Txn_Same_Amt_Same_Merchant',
 'Refund',
 'Total_Refund_Txn_Per_User',
 'Pct_Refund_Txn_Per_User',
 'Same_Amt_Txns',
 'Successful_Abuse_Txns',
 'Total_Transactions_By_User_In_Lifetime',
 'Transactions_In_One_Minute',
 'Transactions_Per_User_Per_Hr',
 'Transactions_Per_User_Per_Day',
 'Transactions_Per_User_Per_Month',
 'Unique_Merchants_Per_User',
 'AverageLoginTime(Hour)',
 'AverageLoginTime(Min)',
 'UsersPerEmail',
 'CountryPrefixMobile_API',
 'UserCountPerSimId ']

In [35]:
drop_list = op[1]

In [36]:
#dropping varibales from main dataframe after univariate analysis
df_xgb = df.drop(columns = drop_list)

In [37]:
#shape of dataframe after univariate analysis
df_xgb.shape

(1485, 91)

In [38]:
'''
WoE and IV calculation
'''

#helper function to calculate WOE and IV values 
def WOE_IV_cal(data):
    #Intermediate steps involved in calculation of WOE and IV
    data['NONEVENT'] = data['COUNT'] - data['EVENT']
    sum_event = data['EVENT'].sum()
    sum_nonevent = data['NONEVENT'].sum()
    #For cases where the value of event or non event is coming out to be zero - add .5 for non-event and event
    for idx in range(0,data.shape[0]):
        if data.iloc[idx,2] == 0 or data.iloc[idx,3] == 0:
            data.iloc[idx,2] = data.iloc[idx,2] + 0.5
            data.iloc[idx,3] = data.iloc[idx,3] + 0.5
    data['%EVENT'] = (data['EVENT']/sum_event) * 100
    data['%NONEVENT'] = (data['NONEVENT']/sum_nonevent) * 100
    data['WOE'] = np.log(data['%NONEVENT']/data['%EVENT'])
    data['IV'] = (data['%NONEVENT']-data['%EVENT'])/100*data['WOE']
    return data

In [39]:
#Function for automated binning process 
def binning(data,variable = None, categorical = False, cuts = None, target = None):
    
    #Scenario where the variable is categorical
    if categorical:
        #Fetching variable column and target column in new dataframe - df  
        df = data[[variable, target]]
        
        #Input for WOE_IV_cal (helper function)
        df[variable] = df[variable].fillna('missing')
        df_input = df.groupby(variable).agg({variable:'count', target:'sum'})\
                     .rename(columns={variable:'COUNT',target:'EVENT'}).reset_index()
        
        df_WOE_IV = WOE_IV_cal(df_input)
        df_WOE_IV.sort_values('WOE', inplace = True)
        
        #Column name for plot
        x = variable
        
    #Scenario where the variable is numeric
    else:
        #Fetching variable column and target column in new dataframe - df  
        df = data[[variable, target]]
        #creation of bins based on cuts passed as input 
        df[variable] = pd.qcut(df[variable], q = cuts, duplicates='drop')
        df[variable] = df[variable].astype(str)

        #Input for WOE_IV_cal (helper function)
        df_input = df.groupby(variable).agg({variable:'count', target:'sum'})\
                     .rename(columns={variable:'COUNT',target:'EVENT'}).reset_index()
        
        #calling WOE_IV_cal (helper function) 
        df_WOE_IV = WOE_IV_cal(df_input)
        
        #Column name for plot
        x = variable
        
    #Bar plot to distribution of WOE
    fig = px.histogram(df_WOE_IV, x=x, y='WOE')
    fig.update_layout(
        yaxis=dict(
        title='WOE',
        )
    )
    
    #Total IV
    tot_IV = df_WOE_IV['IV'].sum()
    return df_WOE_IV, fig, tot_IV

In [40]:
#filtering categorical columns based on type
type_dic = dict(df_xgb.dtypes)
cat_list = []
for col, col_type in type_dic.items():
    if col_type == 'object':
        cat_list.append(col)

In [41]:
#list of categorical variables 
cat_list

['Card Issued Country Code',
 'Association Name',
 'Credit/Debit Card Flag',
 'OS Type',
 'RegionName_GovData',
 'CircleName_GovData',
 'Taluk_GovData',
 'DistrictName_GovData',
 'StateName_GovData',
 'CountryCodeIP_API',
 'CountryNameIP_API',
 'RegionNameIP_API',
 'CityNameIP_API',
 'json',
 'IsValidMobile_API',
 'CountryCodeMobile_API',
 'LocationMobile_API',
 'CarrierMobile_API',
 'LineTypeMobile_API']

In [42]:
df_logreg = df_xgb.copy()

In [43]:
#calculation of iv values for different categorical features
iv_val = {}
iv_table = []
graphs = []
var_grp = pd.DataFrame()
card = pd.DataFrame()
drop_list = []
for i in cat_list:
    op = binning(df_logreg, variable = i,categorical = True, target = 'Abuse status')
    if op[2]<0.02 or op[2]>0.5:
        drop_list.append(i)
    else:
        iv_val[i] = op[2] 
        graphs.append(op[1])
        iv_table.append(op[0])
        temp = op[0][[i,'WOE']]
        df_logreg[i].fillna('missing', inplace = True)
        var_grp = pd.concat([var_grp,df_logreg[i]], axis = 1)
        df_logreg = pd.merge(df_logreg, temp, on = i, how='inner').drop(columns=[i]).rename(columns={'WOE':i})
        temp_1 = op[0]
        label = temp_1.columns[0]
        temp_1['variable'] = label
        temp_1.rename(columns = {label:'var_group'}, inplace=True)
        card = pd.concat([card,temp_1])

In [44]:
#iv values for categorical variables
iv_values  = pd.DataFrame.from_dict(iv_val, orient = 'index')
iv_values.columns = ['IV']

In [45]:
#droplist of weak categorical features, which are required to be dropped 
drop_list

['Card Issued Country Code',
 'Association Name',
 'Credit/Debit Card Flag',
 'RegionName_GovData',
 'CircleName_GovData',
 'Taluk_GovData',
 'DistrictName_GovData',
 'StateName_GovData',
 'CountryCodeIP_API',
 'CountryNameIP_API',
 'RegionNameIP_API',
 'CityNameIP_API',
 'json',
 'LocationMobile_API']

In [46]:
#dropping weak categorical fetaures and creating a new dataframe, df_xgb1
df_xgb1 = df_xgb.drop(columns = drop_list)
df_logreg2 = df_logreg.drop(columns = drop_list)

In [47]:
df_xgb2 = df_xgb1.copy()
df_xgb2 = df_xgb2.drop('Txn_ID', 1)

In [48]:
df_logreg3 = df_logreg2.copy()
df_logreg3 = df_logreg2.drop('Txn_ID', 1)

In [49]:
#remaining categorical variables
cat_list = [x for x in cat_list if x not in drop_list]
cat_list

['OS Type',
 'IsValidMobile_API',
 'CountryCodeMobile_API',
 'CarrierMobile_API',
 'LineTypeMobile_API']

In [50]:
#list of numeric columns
col_list = list(df_logreg3.columns)
num_col = [x for x in col_list if x not in cat_list]
num_col.remove('Abuse status')
num_col

['Amount',
 'ECI',
 'card Bin',
 'Settlement Bin',
 'Amount_Range',
 'Score_Txn',
 'EnglishWordsCount_Txn',
 'TotalWordsCount_Txn',
 'PctEnglishWordsCount_Txn',
 'Vowels_Txn',
 'IsGibberish_Txn',
 'Name Length',
 'Max_Amt',
 'Min_Amt',
 'Mean_Amt',
 'Median_Amt',
 'Spcl_Char_Flag_Txn',
 'Debit',
 'Credit',
 'Prepaid',
 'Using_Debit_Card',
 'Using_Credit_Card',
 'Using_Prepaid_Card',
 'Pct_Using_Debit_Card',
 'Pct_Using_Credit_Card',
 'Pct_Using_Prepaid_Card',
 'Status',
 'BadTryCount',
 'Score_Reg',
 'IsGibberish_Reg',
 'FirstNameLen',
 'MiddleNameLen',
 'LastNameLen',
 'TotalNameLength',
 'EnglishWordsCount_Reg',
 'TotalWordsCount_Reg',
 'PctEnglishWordsCount_Reg',
 'Vowels_Reg',
 'IsBdayFlag(Month)',
 'IsBdayFlag(Day)',
 'DeviationFromAvgLoginTime(Hour)',
 'DeviationFromAvgLoginTime(Min)',
 'DeviationLoginTime-Hr',
 'DeviationLoginTime-Min',
 'DomainCount',
 'PostalCodeCounts',
 'PostalCountsPerRef',
 'UsersPerAccount',
 'UserCountsPerIP',
 'UsersPerBlock1',
 'UsersPerBlock2',
 'User

In [51]:
#calculation of iv values for different numeric features
iv_val_num = {}
drop_list = []
for col in num_col:
    flag = 0
    for i in range(20, 2, -1):
        op = binning(df_logreg2, variable = col,categorical = False, cuts = i, target = 'Abuse status')
        temp_df = op[0]
        if temp_df.iloc[-1,0] == 'nan':
            temp_df = temp_df.drop(temp_df.tail(1).index)
        if temp_df.shape[0] > 2:
            if (temp_df['WOE'].is_monotonic_increasing or temp_df['WOE'].is_monotonic_decreasing) and (op[2]>0.02 and op[2]<0.5):
                flag = 1
                iv_table.append(op[0])
                graphs.append(op[1])
                temp = op[0][[col,'WOE']]
                df_logreg2[col] = pd.qcut(df_logreg2[col], q = i,duplicates='drop') 
                df_logreg2[col] = df_logreg2[col].astype(str)
                var_grp = pd.concat([var_grp,df_logreg2[col]], axis = 1)
                df_logreg2 = pd.merge(df_logreg2, temp, on = col, how ='inner').drop(columns=[col]).rename(columns={'WOE':col})
                temp_1 = op[0]
                label = temp_1.columns[0]
                temp_1['variable'] = label
                temp_1.rename(columns = {label:'var_group'}, inplace=True)
                card = pd.concat([card,temp_1])
                break
        else:
            break
    if flag == 0:
        drop_list.append(col)
    else:
        iv_val_num[col] = op[2] 
card = card.reset_index().drop(columns = ['index'])

In [52]:
drop_list

['Amount',
 'ECI',
 'card Bin',
 'Settlement Bin',
 'Amount_Range',
 'Score_Txn',
 'EnglishWordsCount_Txn',
 'TotalWordsCount_Txn',
 'PctEnglishWordsCount_Txn',
 'Vowels_Txn',
 'IsGibberish_Txn',
 'Name Length',
 'Max_Amt',
 'Min_Amt',
 'Mean_Amt',
 'Median_Amt',
 'Spcl_Char_Flag_Txn',
 'Debit',
 'Credit',
 'Prepaid',
 'Using_Debit_Card',
 'Using_Credit_Card',
 'Using_Prepaid_Card',
 'Pct_Using_Debit_Card',
 'Pct_Using_Credit_Card',
 'Pct_Using_Prepaid_Card',
 'Status',
 'BadTryCount',
 'Score_Reg',
 'IsGibberish_Reg',
 'TotalNameLength',
 'EnglishWordsCount_Reg',
 'PctEnglishWordsCount_Reg',
 'IsBdayFlag(Month)',
 'IsBdayFlag(Day)',
 'DeviationFromAvgLoginTime(Hour)',
 'DeviationFromAvgLoginTime(Min)',
 'DeviationLoginTime-Hr',
 'DeviationLoginTime-Min',
 'DomainCount',
 'PostalCountsPerRef',
 'UsersPerAccount',
 'UserCountsPerIP',
 'UsersPerBlock1',
 'UsersPerBlock2',
 'latitude',
 'longitude',
 'ZipcodeIP_API',
 'Mobile_API',
 'LocalFormatMobile_API',
 'IntFormatMobile_API',
 'Users

In [53]:
#Dataset shape after removing weak numeric variables  
df_xgb1.drop(columns = drop_list, inplace = True)
df_xgb1.shape

(1485, 14)

In [54]:
#Dataset shape after removing weak numeric variables  
df_logreg2.drop(columns=drop_list, inplace = True)
df_logreg2.shape

(1461, 14)

In [55]:
#iv values for numerical variables
iv_val_num  = pd.DataFrame.from_dict(iv_val_num , orient = 'index')
iv_val_num.columns = ['IV']

In [56]:
iv_val_num

,IV
FirstNameLen,0.175332
MiddleNameLen,0.020160
LastNameLen,0.116500
TotalWordsCount_Reg,0.080092
Vowels_Reg,0.497278
PostalCodeCounts,0.222101
UsersPerBlock3,0.082408


In [57]:
#iv values for both numeric and categorical columns
iv_vals = iv_values.append(iv_val_num).reset_index().rename(columns={'index' : 'features'})
iv_vals

,features,IV
0,OS Type,0.082095
1,IsValidMobile_API,0.417031
2,CountryCodeMobile_API,0.412056
3,CarrierMobile_API,0.494634
4,LineTypeMobile_API,0.414344
5,FirstNameLen,0.175332
6,MiddleNameLen,0.020160
7,LastNameLen,0.116500
8,TotalWordsCount_Reg,0.080092
9,Vowels_Reg,0.497278


In [58]:
schema = ['VAR_NAME', 'BIN', 'COUNT', 'EVENT', 'NONEVENT', '%EVENT', '%NONEVENT', 'WOE', 'IV']
for i in iv_table:
    display(HTML(i.rename(columns = {'var_group': 'BIN', 'variable': 'VAR_NAME'})[schema]\
                 .reset_index().drop(columns = ['index']).to_html()))

,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,OS Type,Android,1347,668,679,94.751773,87.051282,-0.084763,0.006527
1,OS Type,ios,114,33,81,4.680851,10.384615,0.796845,0.045450
2,OS Type,missing,24,4,20,0.567376,2.564103,1.508342,0.030117


,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,IsValidMobile_API,missing,1285,682.0,603.0,96.737589,77.307692,-0.224209,0.043563
1,IsValidMobile_API,False,2,0.5,2.5,0.070922,0.320513,1.508342,0.003765
2,IsValidMobile_API,True,198,23.0,175.0,3.262411,22.435897,1.928196,0.369702


,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,CountryCodeMobile_API,,1287,682,605,96.737589,77.564103,-0.220897,0.042354
1,CountryCodeMobile_API,IN,198,23,175,3.262411,22.435897,1.928196,0.369702


,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,CarrierMobile_API,missing,1298,683.0,615.0,96.879433,78.846154,-0.205969,0.037143
1,CarrierMobile_API,Bharat Sanchar Nigam Ltd (BSNL),16,8.0,8.0,1.134752,1.025641,-0.101096,0.000110
2,CarrierMobile_API,Vodafone Idea Ltd (formerly Idea Cellular Ltd),25,5.0,20.0,0.709220,2.564103,1.285198,0.023839
3,CarrierMobile_API,Vodafone Idea Ltd (formerly Vodafone India Ltd),33,5.0,28.0,0.709220,3.589744,1.621670,0.046713
4,CarrierMobile_API,Reliance Communications Ltd (RCOM),4,0.5,4.5,0.070922,0.576923,2.096128,0.010606
5,CarrierMobile_API,Telenor (India) Communications Pvt. Ltd,4,0.5,4.5,0.070922,0.576923,2.096128,0.010606
6,CarrierMobile_API,Reliance Jio Infocomm Ltd (RJIL),31,2.0,29.0,0.283688,3.717949,2.573053,0.088365
7,CarrierMobile_API,Bharti Airtel Ltd,47,2.0,45.0,0.283688,5.769231,3.012419,0.165248
8,CarrierMobile_API,Aircel Cellular Ltd,11,0.5,11.5,0.070922,1.474359,3.034398,0.042586
9,CarrierMobile_API,Tata Teleservices Ltd (TTSL),16,0.5,16.5,0.070922,2.115385,3.395411,0.069418


,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,LineTypeMobile_API,missing,1287,682.0,605.0,96.737589,77.564103,-0.220897,0.042354
1,LineTypeMobile_API,mobile,188,23.0,165.0,3.262411,21.153846,1.869355,0.334454
2,LineTypeMobile_API,landline,10,0.5,10.5,0.070922,1.346154,2.943426,0.037536


...

,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,LastNameLen,"(-0.001, 4.0]",421,248,173,35.177305,22.179487,-0.461233,0.059950
1,LastNameLen,"(4.0, 5.0]",443,211,232,29.929078,29.743590,-0.006217,0.000012
2,LastNameLen,"(5.0, 6.0]",301,133,168,18.865248,21.538462,0.132519,0.003543
3,LastNameLen,"(6.0, 20.0]",320,113,207,16.028369,26.538462,0.504235,0.052996


,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,TotalWordsCount_Reg,"(0.999, 2.0]",1235,620,615,87.943262,78.846154,-0.109193,0.009933
1,TotalWordsCount_Reg,"(2.0, 3.0]",223,81,142,11.489362,18.205128,0.460282,0.030911
2,TotalWordsCount_Reg,"(3.0, 7.0]",27,4,23,0.567376,2.948718,1.648104,0.039247


,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,Vowels_Reg,"(-0.001, 3.0]",378,288,90,40.851064,11.538462,-1.264247,0.370584
1,Vowels_Reg,"(3.0, 4.0]",423,173,250,24.539007,32.051282,0.267073,0.020063
2,Vowels_Reg,"(4.0, 5.0]",313,113,200,16.028369,25.641026,0.469833,0.045163
3,Vowels_Reg,"(5.0, 16.0]",371,131,240,18.581560,30.769231,0.504345,0.061468


,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,PostalCodeCounts,"(0.999, 2.0]",1153,505,648,72.039943,85.263158,0.168522,0.022284
1,PostalCodeCounts,"(2.0, 7.0]",153,72,81,10.271041,10.657895,0.036972,0.000143
2,PostalCodeCounts,"(7.0, 57.0]",155,124,31,17.689016,4.078947,-1.467105,0.199674


,VAR_NAME,BIN,COUNT,EVENT,NONEVENT,%EVENT,%NONEVENT,WOE,IV
0,UsersPerBlock3,"(0.999, 51.0]",277,99,178,14.122682,23.421053,0.505853,0.047036
1,UsersPerBlock3,"(100.0, 128.0]",238,103,135,14.693295,17.763158,0.189735,0.005825
2,UsersPerBlock3,"(128.0, 210.0]",339,168,171,23.965763,22.500000,-0.063111,0.000925
3,UsersPerBlock3,"(210.0, 389.0]",387,211,176,30.099857,23.157895,-0.262185,0.018201
4,UsersPerBlock3,"(51.0, 100.0]",220,120,100,17.118402,13.157895,-0.263132,0.010421


In [59]:
#Function to remove highly correlated two variables 
def bivariate_analysis(df, iv_vals, thresh = 0.8):
    output = []
    df = df.select_dtypes(include=['int64','float64'])
    print(df.shape)
    corr_op = df.corr().abs()
    corr = corr_op.unstack().to_frame().reset_index()
    corr.columns = ['level_0', 'level_1', 'corr']
    corr['correlation'] = np.where(corr['level_0'] == corr['level_1'], 0, corr['corr'])
    corr_1 = corr[corr['correlation'] > 0.9]
    try:
        corr_1['level_2'] = corr_1.apply(lambda x: str(set([x.level_0, x.level_1])), axis=1)
        corr_1.drop_duplicates(subset='level_2',keep ='first', inplace=True)
        final_df = pd.merge(corr_1, iv_vals, left_on = 'level_0', right_on ='features',how='left')
        final_df = pd.merge(final_df, iv_vals, left_on ='level_1', right_on ='features',how='left')
        final_df['drop'] = np.where(final_df['IV_x'] <= final_df['IV_y'], final_df['features_x'], final_df['features_y'])
        output.append(corr_op)
        output.append(corr_1)
        output.append(list(final_df['drop'].unique()))
    except:
        output = [[],[],[]]
    return output

In [60]:
#outputs after bivariate analysis 
op = bivariate_analysis(df_logreg2, iv_vals)

(1461, 14)


In [61]:
#list of variables that are required to dropped based on bivariate analysis 
op[2]

['CountryCodeMobile_API', 'LineTypeMobile_API']

In [62]:
drop_list1 = op[2]
#dropping varibales from main dataframe after bivariate analysis
df_xgb1 = df_xgb1.drop(columns = drop_list1)

In [63]:
drop_list1 = op[2]
#dropping varibales from main dataframe after bivariate analysis
df_logreg2 = df_logreg2.drop(columns = drop_list1)

In [64]:
df_xgb1.shape

(1485, 12)

In [65]:
df_logreg2.shape

(1461, 12)

In [66]:
df_xgb1.columns.to_list()

['Txn_ID',
 'Abuse status',
 'OS Type',
 'FirstNameLen',
 'MiddleNameLen',
 'LastNameLen',
 'TotalWordsCount_Reg',
 'Vowels_Reg',
 'PostalCodeCounts',
 'UsersPerBlock3',
 'IsValidMobile_API',
 'CarrierMobile_API']

In [67]:
df_xgb1.dtypes

Txn_ID                   int64
Abuse status             int64
OS Type                 object
FirstNameLen             int64
MiddleNameLen            int64
LastNameLen              int64
TotalWordsCount_Reg      int64
Vowels_Reg               int64
PostalCodeCounts       float64
UsersPerBlock3           int64
IsValidMobile_API       object
CarrierMobile_API       object
dtype: object

In [68]:
# convert string/object variable to nominal variable
# pre-requisite: variable should be string/object; can have any number of distinct values
# usage: create encoding dictionary for variables; call function with input variable and its encoding
encode_os = {'Android': 1, 'ios': 2}

def create_nominal_vars(df, var_name, encoding):

    df[var_name + '_NOM'] = df[var_name].map(encoding)
    
    return df

df_xgb1 = create_nominal_vars(df_xgb1, 'OS Type', encode_os)

In [69]:
df_xgb1 = df_xgb1.drop(columns = 'OS Type')

In [70]:
'''# convert string/object variable to ordinal variable
# pre-requisite: variable should have more than 2 distinct values
# usage: call function by passing input variable
def create_ordinal_vars(df, var_name):
    df_group = df.groupby(var_name).agg({'Abuse status':'sum', var_name:'count'})
    
    df_group['BAD_RATE'] = (df_group['Abuse status']/df_group[var_name]) * 100
    
    df_group = df_group.rename(columns={var_name:'TOTAL'})
    df_group = df_group.sort_values('BAD_RATE')
    df_group =df_group.reset_index().reset_index().rename(columns={'index':var_name + '_ORD'})
    df_group[var_name + '_ORD'] = df_group[var_name + '_ORD'] + 1
    print(df_group)
    print(list(df_group[var_name]))
    
    var_rank_list = list(df_group[var_name + '_ORD'])
    var_list = list(df_group[var_name])

    df[var_name + '_ORD'] = None
    
    for new, old in zip(var_rank_list, var_list):
        df[var_name + '_ORD'] = np.where(df[var_name] == old, new, df[var_name + '_ORD'])
    
    return df

df_encode_list = df_xgb1.loc[:,df_xgb1.dtypes == np.object]
encode_list = df_encode_list.columns
encode_list

for a in encode_list:
    df_xgb1 = create_ordinal_vars(df_xgb1, a)

df_xgb1 = df_xgb1.drop(encode_list, axis = 1)'''

"# convert string/object variable to ordinal variable\n# pre-requisite: variable should have more than 2 distinct values\n# usage: call function by passing input variable\ndef create_ordinal_vars(df, var_name):\n    df_group = df.groupby(var_name).agg({'Abuse status':'sum', var_name:'count'})\n    \n    df_group['BAD_RATE'] = (df_group['Abuse status']/df_group[var_name]) * 100\n    \n    df_group = df_group.rename(columns={var_name:'TOTAL'})\n    df_group = df_group.sort_values('BAD_RATE')\n    df_group =df_group.reset_index().reset_index().rename(columns={'index':var_name + '_ORD'})\n    df_group[var_name + '_ORD'] = df_group[var_name + '_ORD'] + 1\n    print(df_group)\n    print(list(df_group[var_name]))\n    \n    var_rank_list = list(df_group[var_name + '_ORD'])\n    var_list = list(df_group[var_name])\n\n    df[var_name + '_ORD'] = None\n    \n    for new, old in zip(var_rank_list, var_list):\n        df[var_name + '_ORD'] = np.where(df[var_name] == old, new, df[var_name + '_ORD']

In [71]:
df_xgb1.to_csv('output/xgboost_modeling_data.csv', index=False)

In [72]:
df_logreg2.to_csv('output/logreg_modeling_data.csv', index=False)

In [73]:
df_xgb1 = pd.read_csv('output/xgboost_modeling_data.csv')

In [74]:
df_logreg2 = pd.read_csv('output/logreg_modeling_data.csv')

In [75]:
df_xgb1.dtypes

Txn_ID                   int64
Abuse status             int64
FirstNameLen             int64
MiddleNameLen            int64
LastNameLen              int64
TotalWordsCount_Reg      int64
Vowels_Reg               int64
PostalCodeCounts       float64
UsersPerBlock3           int64
IsValidMobile_API       object
CarrierMobile_API       object
OS Type_NOM            float64
dtype: object

In [97]:
df_logreg2.dtypes

Txn ID                   int64
Abuse status             int64
OS Type                float64
IsValidMobile_API      float64
CarrierMobile_API      float64
FirstNameLen           float64
MiddleNameLen          float64
LastNameLen            float64
TotalWordsCount_Reg    float64
Vowels_Reg             float64
PostalCodeCounts       float64
UsersPerBlock3         float64
dtype: object

In [98]:
feature_drop_list = ['CarrierMobile_API']
feature_include_list = ['IsGibberish_Reg',
                        'IsGibberish_Txn']

df_xgb1 = df_xgb1.rename(columns = {'Txn_ID': 'Txn ID'})
df_logreg2 = df_logreg2.rename(columns = {'Txn_ID': 'Txn ID'})

df_feat_incl = df_merged_original[['Txn ID'] + feature_include_list]

df_xgb_final = df_xgb1.merge(df_feat_incl, on = 'Txn ID', how = 'left')
df_logreg_final = df_logreg2.merge(df_feat_incl, on = 'Txn ID', how = 'left')

df_xgb_final = df_xgb_final.drop(columns = feature_drop_list)
df_logreg_final = df_logreg_final.drop(columns = feature_drop_list)

In [99]:
df_xgb_final.shape

(1485, 13)

In [100]:
df_logreg_final.shape

(1461, 13)

In [101]:
df_xgb_final.dtypes

Txn ID                   int64
Abuse status             int64
FirstNameLen             int64
MiddleNameLen            int64
LastNameLen              int64
TotalWordsCount_Reg      int64
Vowels_Reg               int64
PostalCodeCounts       float64
UsersPerBlock3           int64
IsValidMobile_API       object
OS Type_NOM            float64
IsGibberish_Reg          int64
IsGibberish_Txn          int64
dtype: object

In [102]:
df_logreg_final.dtypes

Txn ID                   int64
Abuse status             int64
OS Type                float64
IsValidMobile_API      float64
FirstNameLen           float64
MiddleNameLen          float64
LastNameLen            float64
TotalWordsCount_Reg    float64
Vowels_Reg             float64
PostalCodeCounts       float64
UsersPerBlock3         float64
IsGibberish_Reg          int64
IsGibberish_Txn          int64
dtype: object

In [103]:
'''
xgboost model
'''

x = df_xgb_final
y = df_xgb_final["Abuse status"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [104]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1188, 13), (297, 13), (1188,), (297,))

In [105]:
x_train1 = x_train.drop('Abuse status', axis = 1)
x_test1 = x_test.drop('Abuse status', axis = 1)

x_train1 = x_train1.drop("Txn ID", axis=1)
x_test1 = x_test1.drop("Txn ID", axis=1)

In [106]:
x_train1.shape, y_train.shape, x_test1.shape, y_test.shape

((1188, 11), (1188,), (297, 11), (297,))

In [107]:
xg_xtrain = x_train1.values
xg_ytrain = y_train.values
xg_xtest = x_test1.values
xg_ytest = y_test.values

xgboost = XGBClassifier(learning_rate = 0.02,
                        n_estimators = 600,
                        objective = 'binary:logistic',
                        silent = True,
                        nthread = 1)

In [108]:
xgboost.fit(xg_xtrain, xg_ytrain)


ytrain_pred = xgboost.predict(xg_xtrain)
ytrain_pred_prob = xgboost.predict_proba(xg_xtrain)[:,1]

ytest_pred = xgboost.predict(xg_xtest)
ytest_pred_prob = xgboost.predict_proba(xg_xtest)[:,1]

[20:57:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:57:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [109]:
feat_imp = pd.DataFrame(xgboost.feature_importances_, index = x_train1.columns)
feat_imp = feat_imp.reset_index()
feat_imp.columns = ['Feature', 'Feature_Importance']
feat_imp = feat_imp.sort_values(by = 'Feature_Importance', ascending = False)
feat_imp['Cumm_Feat_Imp'] = feat_imp['Feature_Importance'].cumsum()

f = 'gain'
feat_gain = pd.DataFrame.from_dict(xgboost.get_booster().get_score(importance_type = f), orient = 'index')
feat_gain = feat_gain.reset_index()
feat_gain.columns = ['Feature', 'Gain']

feat_imp = feat_imp.merge(feat_gain, on = 'Feature', how = 'left')
feat_imp = feat_imp.sort_values('Gain', ascending = False)

In [110]:
feat_imp

,Feature,Feature_Importance,Cumm_Feat_Imp,Gain
0,IsGibberish_Reg,0.501534,0.501534,NaN
1,IsValidMobile_API,0.158338,0.659872,NaN
2,IsGibberish_Txn,0.096254,0.756126,NaN
3,Vowels_Reg,0.053509,0.809636,NaN
4,PostalCodeCounts,0.044124,0.853760,NaN
5,MiddleNameLen,0.027077,0.880836,NaN
6,OS Type_NOM,0.026201,0.907038,NaN
7,FirstNameLen,0.025833,0.932871,NaN
8,UsersPerBlock3,0.022632,0.955503,NaN
9,TotalWordsCount_Reg,0.022256,0.977759,NaN


In [111]:
train_acc_xgb = round(xgboost.score(xg_xtrain, xg_ytrain) * 100, 2)
train_acc_xgb

85.52

In [112]:
test_acc_xgb = round(xgboost.score(xg_xtest, xg_ytest) * 100, 2)
test_acc_xgb

66.67

In [113]:
xgboost_train_output = x_train.copy()
xgboost_test_output = x_test.copy()

In [114]:
xgboost_train_output['Predicted_Abuse_Status_Class'] = ytrain_pred
xgboost_train_output['Predicted_Abuse_Status_Probability'] = ytrain_pred_prob

In [115]:
xgboost_test_output['Predicted_Abuse_Status_Class'] = ytest_pred
xgboost_test_output['Predicted_Abuse_Status_Probability'] = ytest_pred_prob

In [117]:
xgboost_report = classification_report(y_test, ytest_pred, labels = [0, 1])

print(xgboost_report)

              precision    recall  f1-score   support

           0       0.62      0.74      0.68       140
           1       0.72      0.60      0.66       157

    accuracy                           0.67       297
   macro avg       0.67      0.67      0.67       297
weighted avg       0.68      0.67      0.67       297



In [118]:
'''
logistic regression model - for benchmark
'''

x1 = df_logreg2
y1 = df_logreg2["Abuse status"]
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size = 0.2, random_state = 42)

In [119]:
x1_train.shape, x1_test.shape, y1_train.shape, y1_test.shape

((1168, 12), (293, 12), (1168,), (293,))

In [120]:
x1_train1 = x1_train.drop("Abuse status", axis=1)
x1_train1 = x1_train1.drop("Txn ID", axis = 1)
x1_test1 = x1_test.drop("Abuse status", axis=1)
x1_test1 = x1_test1.drop("Txn ID", axis = 1)

x1_train1.shape, y1_train.shape, x1_test1.shape, y1_test.shape

((1168, 10), (1168,), (293, 10), (293,))

In [121]:
logreg = LogisticRegression()
logreg.fit(x1_train1, y1_train)

y1_train_pred = logreg.predict(x1_train1)
y1_train_pred_prob = logreg.predict_proba(x1_train1)

y1_test_pred = logreg.predict(x1_test1)
y1_test_pred_prob = logreg.predict_proba(x1_test1)

In [122]:
acc_log = round(logreg.score(x1_train1, y1_train_pred) * 100, 2)
acc_log

100.0

In [123]:
acc_log = round(logreg.score(x1_test1, y1_test_pred) * 100, 2)
acc_log

100.0

In [124]:
logreg_report = classification_report(y1_test, y1_test_pred, labels = [0, 1])

In [125]:
print(logreg_report)

              precision    recall  f1-score   support

           0       0.63      0.75      0.69       154
           1       0.65      0.51      0.57       139

    accuracy                           0.64       293
   macro avg       0.64      0.63      0.63       293
weighted avg       0.64      0.64      0.63       293



In [126]:
'''
creating pickle file of xgboost model
'''

with open(final_model_pickle, 'wb') as f:
    pickle.dump(xgboost, f)

In [128]:
xgboost_train_output.to_csv(xgboost_train_output_file, index = False)
xgboost_test_output.to_csv(xgboost_test_output_file, index = False)